Before running this notebook, please kindly :
1. Add Pickle Best Model :   
https://drive.google.com/file/d/1ckmqfyU4EgBkYutBXQZqb8D9sByGvms_/view?usp=sharing
2. Add custom_transformer_2.py :   
https://drive.google.com/file/d/1mhAkbzVcFfQ8tEXSDUzTnuSTPO9pN5T1/view?usp=sharing


In [ ]:
#!pip install fastapi
#!pip install category_encoders

In [ ]:
#!pip install "uvicorn[standard]"

    %%writefile custom_transformer.py
    import pandas as pd
    import numpy as np
    from sklearn.base import BaseEstimator, TransformerMixin

    class CleanAgeTransformer(BaseEstimator, TransformerMixin):
        def fit(self, X, y=None):
            return self
        
        def transform(self, X):
            def clean_age_value(val):
                if pd.isna(val):
                    return np.nan
                try:
                    # Convert to string and clean
                    clean_val = str(val).replace('years', '').strip()
                    # Convert to float first, then int to handle decimal values
                    return int(float(clean_val))
                except (ValueError, TypeError):
                    return np.nan
            
            if isinstance(X, pd.DataFrame):
                cleaned = X.iloc[:, 0].apply(clean_age_value)
                return cleaned.to_frame()
            elif isinstance(X, pd.Series):
                cleaned = X.apply(clean_age_value)
                return cleaned.to_frame()
            else:
                cleaned_val = clean_age_value(X)
                return np.array([[cleaned_val]])


In [1]:
%%writefile backend_script.py
from fastapi import FastAPI
import pickle
import pandas as pd
from pydantic import BaseModel
from custom_transformer_2 import CleanAgeTransformer
import category_encoders as ce

with open("best_pipeline_3.pkl", "rb") as f:
  model = pickle.load(f)

app = FastAPI()

class InputData(BaseModel):
    Gender: str
    Age: float
    Height: float
    Weight: float
    family_history_with_overweight: str
    FAVC: str
    FCVC: float
    NCP: float
    CAEC: str
    SMOKE: str
    CH2O: float
    SCC: str
    FAF: float
    TUE: float
    CALC: str
    MTRANS: str

label_map = {
    0: "Insufficient Weight",
    1: "Normal Weight",
    2: "Overweight Level I",
    3: "Overweight Level II",
    4: "Obesity Type I",
    5: "Obesity Type II",
    6: "Obesity Type III"
}

@app.get("/api-endpoint")

def read_root():
  return {"message" : "Welcome to Obesity Model API!"}

@app.post("/predict")

def predict(data: InputData):
  dfInput = pd.DataFrame([data.dict()])
  prediction = model.predict(dfInput)[0]
  prediction_labelled = label_map.get(prediction, "Unknown")
  return prediction_labelled

Overwriting backend_script.py


In [2]:
!npm install localtunnel

⠙⠹⠸⠼
up to date, audited 23 packages in 755ms
⠼
⠼3 packages are looking for funding
⠼  run `npm fund` for details
⠼
2 high severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠴

In [3]:
!curl ipv4.icanhazip.com

34.53.105.180


In [4]:
!uvicorn backend_script:app --reload &>/content/logs.txt &

In [ ]:
#!uvicorn backend_script:app --host 0.0.0.0 --port 8000

In [5]:
!npx localtunnel --port 8000

⠙your url is: https://large-crabs-drive.loca.lt
/content/node_modules/localtunnel/bin/lt.js:81
    throw err;
    ^

Error: connection refused: localtunnel.me:34839 (check your firewall settings)
    at Socket.<anonymous> (/content/node_modules/localtunnel/lib/TunnelCluster.js:52:11)
    at Socket.emit (node:events:524:28)
    at emitErrorNT (node:internal/streams/destroy:169:8)
    at emitErrorCloseNT (node:internal/streams/destroy:128:3)
    at process.processTicksAndRejections (node:internal/process/task_queues:82:21)

Node.js v20.19.0
⠙